In [ ]:
#default_exp synthesizer

# Synthesizer
> Loading, defining and training wavenet based speech synthesizers from various audio representations.

# Imports

In [ ]:
from fastai2.basics import *
import librosa
import librosa.display
import IPython

In [ ]:
file = '/home/cerberus/.fastai/data/es/clips/'

# Loading Data

## Data Types

In [ ]:
class ArraySpec(ndarray):
    #TODO Inherite from ArrayImage
    _show_args = {'cmap':'gist_gray', 'x_axis':'s', 'y_axis':'linear'}
    sr = None
    def __new__(cls, x, sr, *args, **kwargs):
        if isinstance(x,tuple): super().__new__(cls, x, *args, **kwargs)
        if not isinstance(x,ndarray): x = array(x)
        x = x.view(cls); x.sr = sr
        return x
    
    def show(self, ctx=None, figsize=None, **kwargs):
        return librosa.display.specshow(abs(self), **{**self._show_args, **kwargs})

In [ ]:
class ArraySignal(ndarray):
    _show_args = {'x_axis':'s'}
    sr = None
    def __new__(cls, x, sr, *args, **kwargs):
        if isinstance(x,tuple): super().__new__(cls, x, *args, **kwargs)
        if not isinstance(x,ndarray): x = array(x)
        x = x.view(cls); x.sr = sr
        return x
    
    def show(self, ctx=None, figsize=None, **kwargs):
        librosa.display.waveplot(self, **{**self._show_args, **kwargs})
        return IPython.display.Audio(self, rate=self.sr)

In [ ]:
class TensorSpec(TensorBase):
    _show_args = ArraySpec._show_args
    def show(self, ctx=None, figsize=None, **kwargs):
        return librosa.display.specshow(abs(self.cpu().numpy()), **{**self._show_args, **kwargs})

In [ ]:
class TensorSignal(TensorBase):
    _show_args = ArraySignal._show_args
    def show(self, ctx=None, figsize=None, **kwargs):
        data = self.cpu().numpy()
        librosa.display.waveplot(abs(data), **{**self._show_args, **kwargs})
        return IPython.display.Audio(data, rate=self.sr)

In [ ]:
ArraySpec._tensor_cls = TensorSpec
ArraySignal._tensor_cls = TensorSignal

In [ ]:
@ToTensor
def encodes(self, o:ArraySpec): return ArraySpec._tensor_cls(abs(o))

@ToTensor
def encodes(self, o:ArraySignal): return ArraySignal._tensor_cls(o)

## DataBlock

In [ ]:
audio_extensions = set(k for k,v in mimetypes.types_map.items() if v.startswith('audio/'))
def get_audio_files(path, recurse=True, folders=None):
    "Get image files in `path` recursively, only in `folders`, if specified."
    return get_files(path, extensions=audio_extensions, recurse=recurse, folders=folders)

In [ ]:
class Spectify(Transform):
    #TODO Allow parameter changes
    def __init__(self, func=librosa.stft, **kwargs):
        self.transform = partial(func, **kwargs)
    def encodes(self, file:(str, Path)):
        audio, sr = librosa.load(file)
        return ArraySpec(self.transform(audio), sr)
    def encodes(self, audio:ndarray, sr):
        return ArraySpec(self.transform(audio), sr)

In [ ]:
class AudioLoader(Transform):
    @delegates(librosa.load)
    def __init__(self, sr=22050, **kwargs):
        store_attr(self, 'sr, kwargs')
    def encodes(self, file):
        y, sr = librosa.load(file, self.sr, **self.kwargs)
        if self.sr: return ArraySignal(y, self.sr)
        return ArraySignal(y, sr)

In [ ]:
def SpecBlock(spec_func=librosa.stft, **kwargs):
    return TransformBlock(type_tfms=Spectify(spec_func, **kwargs))

In [ ]:
def SignalBlock(sr, **kwargs):
    return TransformBlock(type_tfms=AudioLoader(sr, **kwargs))

## DataLoader

In [ ]:
data = DataBlock((SpecBlock(), SignalBlock(44100)),
                 get_items=get_audio_files,
                 splitter=RandomSplitter())

In [ ]:
dls = data.dataloaders(file, batch_tfms=ToTensor(), bs=4)

/home/cerberus/.local/anaconda3/envs/fai2/lib/python3.8/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')
/home/cerberus/.local/anaconda3/envs/fai2/lib/python3.8/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')
/home/cerberus/.local/anaconda3/envs/fai2/lib/python3.8/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')
/home/cerberus/.local/anaconda3/envs/fai2/lib/python3.8/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')


In [ ]:
dls.one_batch()

/home/cerberus/.local/anaconda3/envs/fai2/lib/python3.8/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')
/home/cerberus/.local/anaconda3/envs/fai2/lib/python3.8/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')
/home/cerberus/.local/anaconda3/envs/fai2/lib/python3.8/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')
/home/cerberus/.local/anaconda3/envs/fai2/lib/python3.8/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')
/home/cerberus/.local/anaconda3/envs/fai2/lib/python3.8/site-packages/librosa/core/audio.py:161: UserWarning: PySoun

RuntimeError: invalid argument 0: Sizes of tensors must match except in dimension 0. Got 191 and 157 in dimension 2 at /pytorch/aten/src/TH/generic/THTensor.cpp:612

# Loading Model

In [ ]:
model = torch.hub.load('nvidia/DeepLearningExamples:torchhub', 'nvidia_waveglow')

Using cache found in /home/cerberus/.cache/torch/hub/nvidia_DeepLearningExamples_torchhub


# Training

In [ ]:
learner = Learner(dls, model, )

# Results